In [66]:
import numpy as np

In [99]:
class Variable:
    def __init__(self):
        return
    
    def evaluation_at(self, val_dict):
        return val_dict[self]
    
    def derivative_at(self, var, val_dict):
        return 1.0 if var is self else 0.0
    
    def __add__(self, another):
        return Expression(Add, self, another)
    
    def __radd__(self, another):
        return Expression(Add, another, self)
    
    def __sub__(self, another):
        return Expression(Sub, self, another)
    
    def __rsub__(self, another):
        return Expression(Sub, another, self)

In [114]:
class Expression:
    def __init__(self, ele_func, sub_expr1, sub_expr2=None):
        self._ele_func  = ele_func
        self._sub_expr1 = sub_expr1
        self._sub_expr2 = sub_expr2
    
    def evaluation_at(self, val_dict):
        
        # self._sub_expr2 is None implies that self._ele_func is an unary operator
        if self._sub_expr2 is None: 
            return self._ele_func.evaluation_at(
                self._sub_expr1, val_dict)
        
        # self._sub_expr2 not None implies that self._ele_func is a binary operator
        else:
            return self._ele_func.evaluation_at(
                self._sub_expr1, self._sub_expr2, val_dict)
    
    def derivative_at(self, var, val_dict):
        
        # sub_expr2 is None implies that _ele_func is an unary operator
        if self._sub_expr2 is None:
            return self._ele_func.derivative_at(
                self._sub_expr1, var, val_dict)
        
        # sub_expr2 not None implies that _ele_func is a binary operator
        else:
            return self._ele_func.derivative_at(
                self._sub_expr1, self._sub_expr2, var, val_dict)
    
    def __add__(self, another):
        return Expression(Add, self, another)
    
    def __radd__(self, another):
        return Expression(Add, another, self)
    
    def __sub__(self, another):
        return Expression(Sub, self, another)
    
    def __rsub__(self, another):
        return Expression(Sub, another, self)

In [115]:
class Add:
    @staticmethod
    def evaluation_at(sub_expr1, sub_expr2, val_dict):
        return sub_expr1.evaluation_at(val_dict) + \
               sub_expr2.evaluation_at(val_dict)
    @staticmethod
    def derivative_at(sub_expr1, sub_expr2, var, val_dict):
        return sub_expr1.derivative_at(var, val_dict) + \
               sub_expr2.derivative_at(var, val_dict)

In [116]:
class Sub:
    @staticmethod
    def evaluation_at(sub_expr1, sub_expr2, val_dict):
        return sub_expr1.evaluation_at(val_dict) - \
               sub_expr2.evaluation_at(val_dict)
    @staticmethod
    def derivative_at(sub_expr1, sub_expr2, var, val_dict):
        return sub_expr1.derivative_at(var, val_dict) - \
               sub_expr2.derivative_at(var, val_dict)

In [122]:
class Exp:
    @staticmethod
    def evaluation_at(sub_expr1, val_dict):
        return np.exp(sub_expr1.evaluation_at(val_dict))
    
    @staticmethod
    def derivative_at(sub_expr1, var, val_dict):
        return sub_expr1.derivative_at(var, val_dict) * np.exp(sub_expr1.evaluation_at(val_dict))

In [135]:
class Functions:
    @staticmethod
    def exp(expr):
        return Expression(Exp, expr)

Let $f(a, b) = e^{a-b+c}$, at $(a, b, c) = (1.0, 2.0, 3.0)$, that should be $e^{2} \approx 7.389$.

In [138]:
a = Variable()
b = Variable()
c = Variable()
f = Functions.exp(a-b+c)

In [139]:
f.evaluation_at({a: 1.0, b: 2.0, c:3.0})

7.38905609893065

$\dfrac{\partial f}{\partial b} = -e^{a-b+c}$, at $(a, b, c) = (1.0, 2.0, 3.0)$, that should be $-e^{2} \approx -7.389$.

In [141]:
f.derivative_at(b, {a: 1.0, b: 2.0, c:3.0})

-7.38905609893065

$\dfrac{\partial f}{\partial a} = e^{a-b}$, at $(a, b) = (1.0, 23.0)$, that should be $e^{2} \approx 7.389$.

In [129]:
f.derivative_at(a, {a: 1.0, b: 3.0})

0.1353352832366127